In [4]:
# uncomment the next line to run it once to install dependencies.
#!pip install dash plotly pandas requests pandas_ta 'numpy<2.0.0'

In [5]:
import requests
import zipfile
import io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from quote_chart import create_chart_app
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas_ta as ta

In [6]:
def get_prices_from_binance():
    print('downloading data from binance...')
    # Download the ZIP file from Binance Vision
    previous_month_date = datetime.now() - relativedelta(months=1)
    formatted_date = previous_month_date.strftime("%Y-%m") # formatted as "YYYY-MM"
    url = f'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-{formatted_date}.zip'
    response = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(response.content))

    # Extract the CSV file from the ZIP archive
    csv_filename = z.namelist()[0]
    csv_file = z.open(csv_filename)

    # Read the CSV file into a DataFrame
    data = pd.read_csv(csv_file, header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

    # Convert the 'timestamp' column to datetime and set it as the index
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)

    # Keep only the required columns
    data = data[['open', 'high', 'low', 'close', 'volume']]

    print('download complete')
    return data

In [7]:
def generate_random_prices():
    # Generate realistic minute data
    np.random.seed(42)

    # Generate datetime index for 30 days of minute data
    dates = pd.date_range('2023-01-01 09:30', periods=30*24*60, freq='min')

    # Initialize data with log returns to simulate realistic price movements
    log_returns = 1 + np.random.normal(0, 0.001, len(dates))
    price = 100 * log_returns.cumprod()  # Starting price of 100

    data = pd.DataFrame(index=dates)
    data['close'] = price
    data['open'] = data['close'].shift(1)

    # Correct data for realism
    data['high'] = data[['open', 'close']].max(axis=1) * (1 + np.abs(np.random.normal(0, 0.001, len(dates))))
    data['low'] = data[['open', 'close']].min(axis=1) * (1 - np.abs(np.random.normal(0, 0.001, len(dates))))
    data['volume'] = np.random.randint(100, 1000, size=len(dates))

    # Round values to two decimal places
    data = data.round(2)
    return data


In [8]:
data = get_prices_from_binance()
# data = generate_random_prices()

downloading data from binance...
download complete


In [9]:
# resampled_df will keep resampled version of data. by default there is no resampling until a period button is pressed.
resampled_df = data.copy()

# called when a period button is pressed under the plot.
def on_period_change(button_id):
    global resampled_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    resampled_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    resampled_df = resampled_df.dropna()
    
# create_figure is called on zoom/pans. it recreates the plot for new x range.
def create_figure(x0, x1):
    if x0 is not None:
        df = resampled_df[x0:x1]
        # uncomment the lines below if you want to slice dataframe so that there will be enough data to plot the 
        # chart and also have data on the left and right so that when the user starts to zoom/pan he will see data.
        #delta = x1 - x0
        #df = candles_df[x0-delta:x1+delta]
        if len(df) == 0:
            df = resampled_df[-100:]
    else:
        # by default (before zoom/pan) show last 100 candles.
        df = resampled_df[-100:]
    # the rest of the code in this function is regular code for plotly.
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]])
    # extended dataframe for technical analysis. 26 and 12 for MACD.
    ex = resampled_df.iloc[max(0, resampled_df.index.get_loc(df.index[0]) - 26 - 12):resampled_df.index.get_loc(df.index[-1])]
    ema21 = ta.ema(ex['close'], 21)[-len(df):]
    fig.add_trace(go.Scatter(x=df.index, y=ema21, mode='lines', line=dict(color='blue'), name='EMA21'), row=1, col=1)
    empty_df = pd.DataFrame(columns=[None, None, None])
    macd = ta.macd(ex['close'], fast=12, slow=26, signal=9, min_periods=None, append=True)[-len(df):] if len(df) > 38 else empty_df
    fig.add_trace(go.Scatter(x=df.index, y=macd[macd.columns[0]], mode='lines', line=dict(color='blue'), name='MACD signal'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=macd[macd.columns[2]], mode='lines', line=dict(color='red'), name='MACD line'), row=3, col=1)
    fig.add_trace(go.Bar(x=df.index, y=macd[macd.columns[1]], name='MACD histogram'), row=3, col=1)
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Prices'), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume', marker=dict(color='orange')), row=2, col=1)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=3, r=3, t=3, b=3),
        yaxis=dict(side='right'),
        yaxis3=dict(side='right'),
        yaxis5=dict(side='right'),
        legend=dict(
            y=0.97,
            x=0.97,
        ),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

# create dash app and run it.
app = create_chart_app(create_figure, on_period_change, period_buttons=None, debug=False)
app.run_server(debug=True)
